In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp submission

<IPython.core.display.Javascript object>

# Submission

In [ ]:
#hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [ ]:
import uuid
import numpy as np
import pandas as pd
from pathlib import Path
from typeguard import typechecked
from abc import ABC, abstractmethod
from rich import print as rich_print
from numerapi import NumerAPI, SignalsAPI

from numerai_blocks.key import Key

<IPython.core.display.Javascript object>

## 1. Base

In [ ]:
@typechecked
class BaseSubmittor(ABC):
    def __init__(self, directory_path: str, key: Key):
        self.dir = Path(directory_path)
        self._create_directory()
        self.key = key

    @abstractmethod
    def save_csv(self, dataf: pd.DataFrame, file_name: str, target_columns: list, *args, **kwargs):
        """ Save DataFrame with 'id' + 'target_columns'. """
        ...

    @abstractmethod
    def upload_predictions(self, file_name: str, model_name: str, *args, **kwargs):
        """ Upload file for given model name through API. """
        ...

    def full_submission(self, dataf: pd.DataFrame, file_name: str, model_name: str, target_columns: list, *args, **kwargs):
        """ Save DataFrame and upload predictions through API. """
        self.save_csv(dataf=dataf, file_name=file_name, target_columns=target_columns, *args, **kwargs)
        self.upload_predictions(file_name=file_name, model_name=model_name, *args, **kwargs)

    def _create_directory(self):
        """ Create base directory if it does not exist. """
        if not self.dir.is_dir():
            rich_print(f"No existing directory found at '[blue]{self.dir}[/blue]'. Creating directory...")
            self.dir.mkdir(parents=True, exist_ok=True)

    def __call__(self, dataf: pd.DataFrame, file_name: str, model_name: str, target_columns: list, *args, **kwargs):
        """
        The most common use case will be to create a CSV and submit it immediately after that.
        full_submission handles this.
        """
        self.full_submission(dataf=dataf,
                             file_name=file_name,
                             model_name=model_name,
                             target_columns=target_columns,
                             *args, **kwargs)

<IPython.core.display.Javascript object>

## 2. Numerai Classic

In [ ]:
class NumeraiClassicSubmittor(BaseSubmittor):
    """
    Submit for Numerai Classic through NumerAPI
    :param directory_path: Base directory to save and read prediction files from.
    :param key: Key object (numerai-blocks.key.Key) containing valid credentials for Numerai Classic.
    *args, **kwargs will be passed to NumerAPI initialization.
    """
    def __init__(self, directory_path: str, key: Key, *args, **kwargs):
        super(NumeraiClassicSubmittor, self).__init__(directory_path=directory_path, key=key)
        self.napi = NumerAPI(public_id=self.key.pub_id, secret_key=self.key.secret_key, *args, **kwargs)

    def save_csv(self, dataf: pd.DataFrame, file_name: str, target_columns: list, *args, **kwargs):
        full_path = str(self.dir / file_name)
        rich_print(f":page_facing_up: Saving predictions CSV to '{full_path}'. :page_facing_up:")
        dataf[target_columns].to_csv(full_path, *args, **kwargs)

    def upload_predictions(self, file_name: str, model_name: str, *args, **kwargs):
        full_path = str(self.dir / file_name)
        model_id = self._get_model_id(model_name=model_name)
        rich_print(f":airplane: Uploading predictions from '{full_path}' for model [bold blue]'{model_name}'[/bold blue] (model_id='{model_id}') :airplane:")
        self.napi.upload_predictions(file_path=full_path,
                                     model_id=model_id,
                                     *args, **kwargs)
        rich_print(f":thumbs_up: Submission of '{full_path}' for [bold blue]{model_name}[/bold blue] is successful! :thumbs_up:")

    def _get_model_id(self, model_name: str) -> str:
        mapping = self.get_model_mapping()
        return mapping[model_name]

    @property
    def get_model_mapping(self) -> dict:
        return self.napi.get_models()

<IPython.core.display.Javascript object>

In [ ]:
# Initialization (Random credentials)
key = Key(pub_id="UFVCTElDX0lE", secret_key="U1VQRVJfU0VDUkVUX0tFWQ==")
num_sub = NumeraiClassicSubmittor(directory_path="test_sub",
                                  key=key)
assert num_sub.dir.is_dir()

# Create random dataframe
n_rows, n_columns = 100, 20
targets = [f"target_{i}" for i in range(n_columns)]
dataf = pd.DataFrame(np.random.uniform(size=(n_rows, n_columns)), columns=targets)
dataf['id'] = [uuid.uuid4() for _ in range(n_rows)]
dataf = dataf.set_index('id')
dataf.head(1)

🔑 Numerai Auth key initialized with pub_id = 'UFVCTElDX0lE' 🔑

,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19
id,,,,,,,,,,,,,,,,,,,,
f6573b07-5160-4f80-ac0b-1073d74df4df,0.57029,0.547674,0.639208,0.478428,0.039853,0.561924,0.627814,0.722491,0.357301,0.599959,0.806091,0.51567,0.342018,0.448455,0.871416,0.599185,0.817916,0.791231,0.422417,0.367146


<IPython.core.display.Javascript object>

In [ ]:
# Save CSV and upload
num_sub.full_submission(dataf=dataf, file_name='test.csv',
                        target_columns=targets, model_name="test")

📄 Saving predictions CSV to 'test_sub/test.csv'. 📄

2022-01-05 23:11:54,600 ERROR numerapi.base_api: Your session is invalid or has expired.


ValueError: Your session is invalid or has expired.

<IPython.core.display.Javascript object>

In [ ]:
pd.read_csv("test_sub/test.csv").head(2)

,id,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,...,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19
0,f6573b07-5160-4f80-ac0b-1073d74df4df,0.570290,0.547674,0.639208,0.478428,0.039853,0.561924,0.627814,0.722491,0.357301,...,0.806091,0.515670,0.342018,0.448455,0.871416,0.599185,0.817916,0.791231,0.422417,0.367146
1,23319e09-7440-44ec-9cd1-a84d2f96f84a,0.857674,0.855701,0.256027,0.639762,0.662072,0.326167,0.332871,0.073530,0.969721,...,0.197043,0.656971,0.712644,0.506917,0.567182,0.855523,0.927963,0.171016,0.504955,0.998586


<IPython.core.display.Javascript object>

## 3. Numerai Signals

<IPython.core.display.Javascript object>

In [ ]:
#hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script; notebook2script()

Converted 00_download.ipynb.
Converted 01_dataloaders.ipynb.
Converted 02_dataset.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04a_model.ipynb.
Converted 04b_modelpipeline.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_prediction_dataset.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>